## IAA Laboratori 3 - Compressing Data via Dimensionality Reduction

In [1]:
%pip install plotly numpy==1.25 nbformat umap-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Overview

- [Maximum Variance Formulation](#Maximum-Variance-Formulation)
  - [scikit-learn Comparisson](#Sklearn-Comparisson)
- [Unsupervised dimensionality reduction via principal component analysis](#Unsupervised-dimensionality-reduction-via-principal-component-analysis-128)
  - [The main steps behind principal component analysis](#The-main-steps-behind-principal-component-analysis)
  - [Extracting the principal components step-by-step](#Extracting-the-principal-components-step-by-step)
  - [Total and explained variance](#Total-and-explained-variance)
  - [Feature transformation](#Feature-transformation)
  - [Principal component analysis in scikit-learn](#Principal-component-analysis-in-scikit-learn)
- [Supervised data compression via linear discriminant analysis](#Supervised-data-compression-via-linear-discriminant-analysis)
  - [Principal component analysis versus linear discriminant analysis](#Principal-component-analysis-versus-linear-discriminant-analysis)
  - [LDA via scikit-learn](#LDA-via-scikit-learn)
- [Projections with Multi-dimensional Scaling (MDS)](#Projections-with-Multi-dimensional-Scaling-(MDS))
- [Projections with Uniform Manifold Approximation and Projection (UMAP)](#Projections-with-Uniform-Manifold-Approximation-and-Projection-(UMAP))
- [Projections with t-distributed Stochastic Neighbor Embedding (t-SNE)](#Projections-with-t-distributed-Stochastic-Neighbor-Embedding-(t-SNE)) 
- [Comparison of Manifold Learning methods](#Comparison-of-Manifold-Learning-methods)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig
from numpy.random import randn, rand, seed
from sklearn.datasets import load_iris
from IPython.display import Image
%matplotlib inline

In [ ]:
%config InlineBackend.figure_format = "retina"
np.set_printoptions(precision=8, suppress=True)

# Maximum Variance Formulation

Let $\mathcal D = \{{\bf x}_n | {\bf x} \in \mathbb{R}^D\}_{n=1}^N$ be a dataset. We seek to **maximize the variance of the projected data**.

Consider the following optimization problem 

$$
\begin{aligned}
    \max_{{\bf u}_1} &\ {{\bf u}_1^T{\bf S}{\bf u}_1} \\
    \text{s.t.} &\ {\bf u}_1^T{\bf u}_1 = 1
\end{aligned} 
$$

Where
$$
    {\bf S} = \frac{1}{N}\sum_{n=1}^N\big({\bf x}_n - \bar{\bf x}\big)\big({\bf x}_n - \bar{\bf x}\big)^T
$$

In [ ]:
seed(314)
nsamples = 20
x1 = np.linspace(0, 5, nsamples) + rand(nsamples) * 0.5
x2 = 3 * x1 + 5 + randn(nsamples) * 2
X = np.c_[x1, x2]
X_bar = X.mean(axis=0, keepdims=True)
X = X - X_bar
plt.scatter(*X.T);

In [ ]:
S = np.einsum("ij,ik->jk", X, X) / nsamples
eigvals, eigvects = eig(S)

In [ ]:
l = eigvals.argmax()
u1 = eigvects[:, [l]]

X_proj = X @ u1

plt.scatter(X_proj, np.ones(nsamples), s=10)
plt.title("Projected Data", fontsize=15);

### scikit-learn Comparisson

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(X)
np.c_[X_proj, pca.transform(X)]

### Proposition

The linear projection onto an $M$-dimensional subspace that maximizes the variance of the projected data is defined by the $M$ eigenvectors of the data covariance matrix $S$, corresponding to the $M$ largest eigenvalues

In [ ]:
l1 = eigvals.argmax()
l2 = eigvals.argmin()

u1 = eigvects[:, [l1]]
u2 = eigvects[:, [l2]]

u1.T @ u2

### An example: The iris dataset

In [ ]:
iris = load_iris()
X, y = iris["data"], iris["target"]
nsamples = len(X)

S = np.einsum("ij,ik->jk", X, X) / nsamples
eigvals, eigvects = eig(S)

# Sort in descending order
l = (-eigvals).argsort()

In [ ]:
u = eigvects[:, l[:2]]
X_proj = X @ u

plt.scatter(*X_proj.T, c=y, cmap="Set1")
plt.title("Principal Subspace", fontsize=15)
plt.xlabel(r"${\bf u}_1$", fontsize=15)
plt.ylabel(r"${\bf u}_2$", fontsize=15);

# Unsupervised dimensionality reduction via principal component analysis (PCA)


## The main steps behind principal component analysis

Load and return the digits dataset (classification).

Each datapoint is a 8x8 image of a digit.

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits(n_class=3)
X, y = digits.data, digits.target
n_samples, n_features = X.shape
n_neighbors = 30
import matplotlib.pyplot as plt
plt.gray()
plt.matshow(digits.images[0])
plt.show()
X.shape


Tensorflow version

Each datapoint is a 28x28 image of a digit.

In [ ]:
#%pip install tensorflow
#from tensorflow.keras.datasets import mnist
#data = mnist.load_data()
#train, test = data
#Xtrain, ytrain = train
#Xtrain.shape

<div class="alert alert-success">
PLAYTIME: Load the full MNIST dataset and repeat the analysis with full dataset
</div>


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=10, ncols=10, figsize=(6, 6))
for idx, ax in enumerate(axs.ravel()):
    ax.imshow(X[idx].reshape((8, 8)), cmap=plt.cm.binary)
    ax.axis("off")
_ = fig.suptitle("A selection from the 64-dimensional (8x8) digits dataset", fontsize=16)

In [ ]:
X_sample = X[y == 0]
X_sample = X_sample.reshape(-1,8,8)
N, M, M = X_sample.shape

Xbar = X_sample.mean(axis=0)
Xhat = (X_sample - Xbar).reshape(N, -1)
X_sample.shape

In [ ]:
%%time
S = np.einsum("ij,ik->jk", Xhat, Xhat) / N
eigvals, eigvects = eig(S)

> Because each eigenvector of the covariance matrix is a vector in the original $D$-dimensional space, we can represent the eigenvectors as images of the same size as data points.

In [ ]:
def format_eigval(eigval):
    label = rf"$\lambda={eigval:.2e}"
    label = label.replace("e+0", "\cdot 10^{")
    return label + "}$"

npc = 4 # number of principal components
fig, ax = plt.subplots(1, npc + 1, figsize=(15, 4))


principal_subspace = eigvects[:, :npc].reshape(M, M , npc).real
eigvectors = [format_eigval(li) for li in eigvals[:npc].real]
labels = ["Mean"] + eigvectors
pca_arrays = np.concatenate((Xbar[..., np.newaxis], principal_subspace), axis=-1)

for n, label in enumerate(labels):
    ax[n].imshow(pca_arrays[..., n])
    ax[n].axis("off")
    ax[n].set_title(label)

**The PCA approximation to a data vector ${\bf x}_n$ is given by**

$$
    \tilde{\bf x}_n = \bar{\bf x} + \sum_{m=1}^M\left({\bf x}_n^{\text T}{\bf u}_m - \bar{\bf x}^{\text T}{\bf u}_m\right) {\bf u}_m
$$

In [ ]:
plt.imshow(Xhat[[13]].reshape(M, M))

In [ ]:
%%time
X = X.reshape(-1,8,8)
N, M, M = X.shape
Xbar = X.mean(axis=0)
Xhat = (X - Xbar).reshape(N, -1)


S = np.einsum("ij,ik->jk", Xhat, Xhat) / N
eigvals, eigvects = eig(S)

u = eigvects[:, :2]
proj = Xhat @ u
plt.scatter(*proj.T, c=y, alpha=0.8, cmap="Dark2")

## Extracting the principal components step-by-step

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits(n_class=3)
X, y = digits.data, digits.target
n_samples, n_features = X.shape
n_neighbors = 30

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, 
                     stratify=y,
                     random_state=0)


Standardizing the data.

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
import numpy as np
cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

print('\nEigenvalues \n%s' % eigen_vals)

In [ ]:
tot = sum(eigen_vals)
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 65), var_exp, alpha=0.5, align='center',
        label='Individual explained variance')
plt.step(range(1, 65), cum_var_exp, where='mid',
         label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
# plt.savefig('images/05_02.png', dpi=300)
plt.show()

## Feature transformation

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i])
               for i in range(len(eigen_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eigen_pairs.sort(key=lambda k: k[0], reverse=True)

In [ ]:
w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
               eigen_pairs[1][1][:, np.newaxis]))
print('Matrix W:\n', w)

**Note**
Depending on which version of NumPy and LAPACK you are using, you may obtain the Matrix W with its signs flipped. Please note that this is not an issue: If $v$ is an eigenvector of a matrix $\Sigma$, we have

$$\Sigma v = \lambda v,$$

where $\lambda$ is our eigenvalue,


then $-v$ is also an eigenvector that has the same eigenvalue, since
$$\Sigma \cdot (-v) = -\Sigma v = -\lambda v = \lambda \cdot (-v).$$

In [ ]:
X_train_std[0].dot(w)

In [ ]:
X_train_pca = X_train_std.dot(w)
colors = ['r', 'b', 'g']
markers = ['s', 'x', 'o']

for l, c, m in zip(np.unique(y_train), colors, markers):
    plt.scatter(X_train_pca[y_train == l, 0], 
                X_train_pca[y_train == l, 1], 
                c=c, label=l, marker=m)

plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.tight_layout()
# plt.savefig('images/05_03.png', dpi=300)
plt.show()

## Principal component analysis in scikit-learn

**NOTE**

The following four code cells illustrate how scikit-learn replicates the results from our own PCA implementation:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
pca.explained_variance_ratio_

In [ ]:
plt.bar(range(1, 65), pca.explained_variance_ratio_, alpha=0.5, align='center')
plt.step(range(1, 65), np.cumsum(pca.explained_variance_ratio_), where='mid')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')

plt.show()

In [ ]:
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

In [ ]:
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1])
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot examples by class
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.6, 
                    color=cmap(idx),
                    edgecolor='black',
                    marker=markers[idx], 
                    label=cl)

Training logistic regression classifier using the first 2 principal components.

In [ ]:
from sklearn.linear_model import LogisticRegression

pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

lr = LogisticRegression(multi_class='ovr', random_state=1, solver='lbfgs')
lr = lr.fit(X_train_pca, y_train)

In [ ]:
plot_decision_regions(X_train_pca, y_train, classifier=lr)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.tight_layout()
# plt.savefig('images/05_04.png', dpi=300)
plt.show()

In [ ]:
plot_decision_regions(X_test_pca, y_test, classifier=lr)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.tight_layout()
# plt.savefig('images/05_05.png', dpi=300)
plt.show()

In [ ]:
pca = PCA(n_components=None)
X_train_pca = pca.fit_transform(X_train_std)
pca.explained_variance_ratio_

# Supervised data compression via linear discriminant analysis (LDA)

## Principal component analysis versus linear discriminant analysis

Solve the generalized eigenvalue problem for the matrix $S_W^{-1}S_B$:

## LDA via scikit-learn

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train_std, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(multi_class='ovr', random_state=1, solver='lbfgs')
lr = lr.fit(X_train_lda, y_train)

plot_decision_regions(X_train_lda, y_train, classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.tight_layout()
# plt.savefig('images/05_09.png', dpi=300)
plt.show()

In [ ]:
X_test_lda = lda.transform(X_test_std)

plot_decision_regions(X_test_lda, y_test, classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.tight_layout()
# plt.savefig('images/05_10.png', dpi=300)
plt.show()

# Projections with Multi-dimensional Scaling (MDS)

Multidimensional scaling (MDS) seeks a low-dimensional representation of the data in which the distances respect well the distances in the original high-dimensional space.

In general, MDS is a technique used for analyzing similarity or dissimilarity data. It attempts to model similarity or dissimilarity data as distances in a geometric spaces. The data can be ratings of similarity between objects, interaction frequencies of molecules, or trade indices between countries.

There exists two types of MDS algorithm: metric and non metric. In scikit-learn, the class MDS implements both. In Metric MDS, the input similarity matrix arises from a metric (and thus respects the triangular inequality), the distances between output two points are then set to be as close as possible to the similarity or dissimilarity data. In the non-metric version, the algorithms will try to preserve the order of the distances, and hence seek for a monotonic relationship between the distances in the embedded space and the similarities/dissimilarities.

MDS is not only an effective technique for dimensionality reduction but also for data visualization. It maintains the same clusters and patterns of high-dimensional data in the lower-dimensional space so you can boil down, say, a 5-dimensional dataset to a 3-dimensional dataset which you can interpret much more easily and naturally.

Normally the distance measure used in MDS is the Euclidean distance, however, any other suitable dissimilarity metric can be used when applying MDS.



In [ ]:
from sklearn.manifold import MDS
import plotly.express as px
from sklearn.datasets import load_digits

digits = load_digits(n_class=3)

mds = MDS(n_components=2, n_init=1, max_iter=120, n_jobs=2, normalized_stress="auto")
projections = mds.fit_transform(digits.data)

fig = px.scatter(
    projections, x=0, y=1,
    color=digits.target.astype(str), labels={'color': 'digit'}
)
fig.show()

In [ ]:
stress = mds.stress_
print(stress)

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
dist_manhattan = manhattan_distances(digits.data)
mds = MDS(n_components=2, n_init=1, max_iter=120, n_jobs=2, normalized_stress="auto",dissimilarity='precomputed')
# Get the embeddings
projections = mds.fit_transform(dist_manhattan)


In [ ]:
stress = mds.stress_
print(stress)

In [ ]:
fig = px.scatter(
    projections, x=0, y=1,
    color=digits.target.astype(str), labels={'color': 'digit'}
)
fig.show()

<div class="alert alert-success">
PLAYTIME: Write a function for selecting the number of MDS projection dimensions based on the stress value  
</div>


# Projections with t-distributed Stochastic Neighbor Embedding (T-SNE)

t-SNE (TSNE) converts affinities of data points to probabilities. The affinities in the original space are represented by Gaussian joint probabilities and the affinities in the embedded space are represented by Student’s t-distributions. This allows t-SNE to be particularly sensitive to local structure and has a few other advantages over existing techniques:

- Revealing the structure at many scales on a single map
- Revealing data that lie in multiple, different, manifolds or clusters
- Reducing the tendency to crowd points together at the center

While Isomap, LLE and variants are best suited to unfold a single continuous low dimensional manifold, t-SNE will focus on the local structure of the data and will tend to extract clustered local groups of samples as highlighted on the S-curve example. This ability to group samples based on the local structure might be beneficial to visually disentangle a dataset that comprises several manifolds at once as is the case in the digits dataset.

The Kullback-Leibler (KL) divergence of the joint probabilities in the original space and the embedded space will be minimized by gradient descent. Note that the KL divergence is not convex, i.e. multiple restarts with different initializations will end up in local minima of the KL divergence. Hence, it is sometimes useful to try different seeds and select the embedding with the lowest KL divergence.

The disadvantages to using t-SNE are roughly:
- t-SNE is computationally expensive, and can take several hours on million-sample datasets where PCA will finish in seconds or minutes
- The Barnes-Hut t-SNE method is limited to two or three dimensional embeddings.
- The algorithm is stochastic and multiple restarts with different seeds can yield different embeddings. However, it is perfectly legitimate to pick the embedding with the least error.
- Global structure is not explicitly preserved. This problem is mitigated by initializing points with PCA (using init='pca').


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.datasets import load_digits

digits = load_digits()

tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(digits.data)

fig = px.scatter(
    projections, x=0, y=1,
    color=digits.target.astype(str), labels={'color': 'digit'}
)
fig.show()

In [ ]:
tsne = TSNE(n_components=3, random_state=0)

proj_3d = tsne.fit_transform(digits.data)


fig_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
     color=digits.target.astype(str), labels={'color': 'digit'}
)
fig_3d.update_traces(marker_size=5)

fig_3d.show()


<div class="alert alert-success">
PLAYTIME: Study the values of perplexity and learning rate. How it affects the KL divergence value? How it affects the projection?
Write a function for repeating the t-SNE projection with different perplexity and learning rate values and select the best projection based on the KL divergence value.
</div>

# Projections with Uniform Manifold Approximation and Projection (UMAP)


t-SNE is a popular dimensionality reduction algorithm that arises from probability theory. Simply put, it projects the high-dimensional data points (sometimes with hundreds of features) into 2D/3D by inducing the projected data to have a similar distribution as the original data points by minimizing something called the KL divergence.

Compared to a method like Principal Component Analysis (PCA), it takes significantly more time to converge, but present significantly better insights when visualized. For example, by projecting features of a flowers, it will be able to distinctly group

Just like t-SNE, UMAP is a dimensionality reduction specifically designed for visualizing complex data in low dimensions (2D or 3D). As the number of data points increase, UMAP becomes more time efficient compared to TSNE.

In the example below, we see how easy it is to use UMAP as a drop-in replacement for scikit-learn's manifold.TSNE.

In [ ]:
import plotly.express as px
from sklearn.datasets import load_digits
from umap import UMAP

digits = load_digits()

umap_2d = UMAP(random_state=0)
umap_2d.fit(digits.data)

projections = umap_2d.transform(digits.data)

fig = px.scatter(
    projections, x=0, y=1,
    color=digits.target.astype(str), labels={'color': 'digit'}
)
fig.show()

In [ ]:
umap_3d = UMAP(n_components=3, init='random', random_state=0)
umap_3d.fit(digits.data)
proj_3d = umap_3d.transform(digits.data)

fig_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
     color=digits.target.astype(str), labels={'color': 'digit'}
)
fig_3d.update_traces(marker_size=5)

fig_3d.show()


<div class="alert alert-success">
PLAYTIME: Study the values of n_neighbors and min_dist parameters for UMAP. How it affects the projection?
</div>

# Comparison of Manifold Learning methods

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits(n_class=3)
X, y = digits.data, digits.target
n_samples, n_features = X.shape
n_neighbors = 30

In [ ]:
import numpy as np
from matplotlib import offsetbox
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler


def plot_embedding(X, title):
    _, ax = plt.subplots()
    X = MinMaxScaler().fit_transform(X)

    for digit in digits.target_names:
        ax.scatter(
            *X[y == digit].T,
            marker=f"${digit}$",
            s=60,
            color=plt.cm.Dark2(digit),
            alpha=0.425,
            zorder=2,
        )
    shown_images = np.array([[1.0, 1.0]])  # just something big
    for i in range(X.shape[0]):
        # plot every digit on the embedding
        # show an annotation box for a group of digits
        dist = np.sum((X[i] - shown_images) ** 2, 1)
        if np.min(dist) < 4e-3:
            # don't show points that are too close
            continue
        shown_images = np.concatenate([shown_images, [X[i]]], axis=0)
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r), X[i]
        )
        imagebox.set(zorder=1)
        ax.add_artist(imagebox)

    ax.set_title(title)
    ax.axis("off")

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import (
    MDS,
    TSNE,
    Isomap,
)
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.random_projection import SparseRandomProjection

embeddings = {
    "Random projection embedding": SparseRandomProjection(
        n_components=2, random_state=42
    ),
    "Truncated SVD embedding": TruncatedSVD(n_components=2),
    "Linear Discriminant Analysis embedding": LinearDiscriminantAnalysis(
        n_components=2
    ),
    "Isomap embedding": Isomap(n_neighbors=n_neighbors, n_components=2),
    "MDS embedding": MDS(
        n_components=2, n_init=1, max_iter=120, n_jobs=2, normalized_stress="auto"
    ),
    "t-SNE embeedding": TSNE(
        n_components=2,
        n_iter=500,
        n_iter_without_progress=150,
        n_jobs=2,
        random_state=0,
    ),
    "NCA embedding": NeighborhoodComponentsAnalysis(
        n_components=2, init="pca", random_state=0
    ),
    "UMAP embedding": UMAP(n_components=2, random_state=42),
}


The LinearDiscriminantAnalysis and the NeighborhoodComponentsAnalysis, are supervised dimensionality reduction method, i.e. they make use of the provided labels, contrary to other methods.

The TSNE is initialized with the embedding that is generated by PCA in this example. It ensures global stability of the embedding, i.e., the embedding does not depend on random initialization.

In [ ]:
from time import time

projections, timing = {}, {}
for name, transformer in embeddings.items():
    if name.startswith("Linear Discriminant Analysis"):
        data = X.copy()
        data.flat[:: X.shape[1] + 1] += 0.01  # Make X invertible
    else:
        data = X

    print(f"Computing {name}...")
    start_time = time()
    projections[name] = transformer.fit_transform(data, y)
    timing[name] = time() - start_time

In [ ]:
for name in timing:
    title = f"{name} (time {timing[name]:.3f}s)"
    plot_embedding(projections[name], title)

plt.show()